In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
import re
import json
import spacy
import os
import requests
import sys
import deepl
import collections
import spacy

import pandas as pd
import numpy as np


In [4]:
nlp = spacy.load('en_core_web_sm')

Structuring the text:

The following functions provide a way to get raw text and categorize it into the different figure numbers, the different sentences the subject spoke for each figure as well as storing information like the special position markers associated with the specific sentences they refer to.

In [5]:
raw_text = '''

I/ @ ^ Une sorte de coléoptère avec des grandes ailes
ou ça peut être une sorte de grenouille.
V Une sorte de masque Viking avec quatre yeux.
Une sorte de silhouette.
Un petit personnage avec  une bouche ouverte qui a l’air surpris
et son reflet là.


II/ V Deux personnages en train d’établir un contact ou
un combat parce qu’il y a du sang ou une jambe qui a été coupée.
Des poumons.
^ Une tête de coq avec la crête
Avion de chasse, planeur, feu des réacteurs ;
là aussi ça peut être un plus grand qui envoie des missiles
Des hommes troncs, peut-être irrationnels
ou des espèces de chenilles avec un gros ventre.

III/ ^ Deux guitares avec des formes bizarres, le manche
tordu.
Encore deux personnages face à face bizarrement constitués,
comme du Picasso. Ou deux personnages qui font qu’un.
Deux espèces de missiles ou de torpilles.
Des poumons
ou un papillon.
V Juste la partie d’un vieillard avec sa canne
ou un embryon.
Une espèce de grosse bête, on voit l’intérieur de son corps,
Les mandibules.

IV/ V Je vois encore une sorte de créature bizarre, un colosse,
Avec  les ailes là et des pattes étranges.
^ Deux personnages sous une espèce de burka.
V La tête de la mouche (film).
> Tête de bébé crocodile ou de cochon vu qu’il y a pas d’oreille
ou peut-être une taupe, une sorte de trompe.

V/ ^ Je vois aussi une sorte de mélange entre une chauve-souris
et un escargot, là les pattes sont un peu étranges.
@ Sinon là je vois rien d’autre.
Peut-être deux têtes de personnages en tout petit,
Ils sont dos à dos.
Le guitariste d’ACDC : sa tête avec ses cheveux.

VI/ @ ^ Une libellule un peu bizarre avec des ailes
à demi détruites.
Là, la mâchoire d’une bête avec des crochets à l’intérieur
de sa bouche, mais y a pas d’œil alors c’est peut-être
une plante carnivore.
L’homme sable (dans Spiderman) en train de reprendre
forme humaine.
Sinon c’est tout ce que je vois.
Ou encore des poumons qui sont cachés juste là
ou des sortes de graines de flageolets.
Ou là une sorte de personnage avec quelque chose
pour faire des bonds.
Finalement, c’est peut-être une sorte d’ange
extraterrestre qui va dans une île parce qu’il y a des ailes.
Il descend par là donc finalement c’est peut-être
un enfant qui va venir au monde.

VII/ V Deux espèces d’hippocampes avec une trompe ici ;
ils sont reliés les deux par quelque chose qui est là ;
ils ont l’air d’être congelés parce qu’il y a des stalactites,
comme peut-être des animaux préhistoriques qui ont été congelés.
^ Ou là sinon on peut voir des espèces de personnages
avec des bras en forme de tronçonneuse qui tentent de se rapprocher.
Des soldats de l’armée prussienne avec des pointes sur le casque.
Là on peut voir une espèce de tête avec un groin mais vu
qu’il y a une proéminence ça peut être le bouffon du roi.
Si on intègre le blanc on peut voir un masque.
V Une arme des Mayas ; ou peut-être pas une arme,
un objet Maya pour le culte
ou une espèce de champignon.

VIII/ V Une vue de microscope avec les colorants qu’il faut.
Deux espèces d’insectes qui sont en train de butiner ce qui est là.
Une tête de mouche avec quelque chose qui dégouline (orange).
Des cordes vocales (traits verts).
Un personnage avec des sortes d’antennes, en faisant
abstraction des cordes vocales.
Deux caméléons qui gravissent une montagne ;
on a l’impression qu’ils se serrent une main ;
il y a une transaction ou quelque chose comme ça :
ils donnent de l’argent ou se rechargent en énergie.

IX/ ^ Deux espèces de créatures difformes qui se rapprochent.
Deux narines, peut-être un crane qui est caché par tout le reste.
Une tête de fantôme en partie cachée par le reste
ou une sorte de champ magnétique.
Deux îles ou deux pays qui sont le reflet de l’autre (vert).
Un morceau de jambon de Parme (rose).
Des espèces de racines, comme le logo de Sépultura (traits oranges)
Une espèce de matière symbiotique qui fusionne.
Un personnage avec des ailes (vert).

X/ ^ Ca fait assez art contemporain, comme du Miro.
Deux espèces de petites bestioles qui se battent pour savoir
ce qui est au milieu (noir).
Des pieuvres ou gendarmes extraterrestres qui essayent
de les séparer (bleu).
Deux petits soleils ou comètes (jaune).
Deux chenilles ou la lettre Omega avec une araignée ou
diapason au boût (vert).
Deux sortes d’embryons qui recrachent une matière bleue (rose).
Deux espèces de fourmis avec une flamme (noir+jaune), elles sont
entrain de rattraper les deux choses qui sont là (orange)
ou la moto du Ghostrider.
V Je vois aussi deux yeux avec une bouche bizarre :
le vert c’est les cornes, le bleu la bouche, le gris un rasoir
électrique ou une sorte de tournevis sonique.
Deux personnages avec un ponpon bleu qui sont
entrain de se liquéfier, ils sont peut-être constitués de peau
et d’air (rose).
Les deux fourmis, elles vont essayer de mettre un chapeau
pour les reboucher pour pas qu’elles se dégonflent.
Deux brosses pour le dos ou à cheveux ou un compas
un peu bizarre (orange).
Finalement les deux chenilles il y a leurs projections astrales
qui est entrain de sortir de leur corps.
Deux personnes qui se soutiennent pour pas perdre l’équilibre
et pas tomber là (bleu central).

Choix + : X parce qu’on peut voir beaucoup de choses.
Choix - : II parce que ça a l’air de deux personnes qui sont entrain de se battre à mort, deux gladiateurs. Puis il y a aussi peut-être les espèces d’avions de guerre.


Rq : latences longues sauf à V et VIII
'''

# 1. Organizing the text into the corresponding figures and responses

Since the original text is in French but most of the resources are English, doing the translation after extracting the position markers may be more appropiate since the automatic translation may mess with those caracters, specially the V and mistranslate them. We are using the DeepL free API which allows for 500.000 characters a month.

Another note on DeepL: there are a lot of "figures" mentioned in the target text, that is, the translation. If you look at the prompt section, the llm seems to confuse it by figures as in art figures instead of figures as in human figures. I know that when doing the DeepL API call you can pass a glossary as an argument to the function. Maybe I would be a good idea to make a glossary with more concrete terms if there are synonyms to those semantically ambiguous nouns. Alternative: look for a way to measure the specificity of a word to deal with those kinds of nouns/terms

This part is commented out because there is no need to abuse the API limit, the ouput dictionary was saved as demo_text_preprocess so we can keep working on the different parts of the project.

As you can see, we are doing most of the parsing with regular expressions. The flow of the function is:
- 1st we split the text according to the roman numerals and we extract the Choix and Rq parts.
- 2nd we clean up the response text to avoid unnecesay extralines and we extract the special markers. At this point we have two full text variables: one raw that we will extract the positional markers from and a clean copy we'll send to DeepL.
- 3rd the clean text for each figure goes through the DeepL API and we store the translation for the whole text corresponding to the figure.
- 4th we split the text corresponding to each figure into its individual sentences.

In [6]:
def get_responses(raw_text):
  
  """This function takes plain text, parses the in-text markers to extract the text corresponding to each Rorschach figure,
  runs the responses through the Deepl API to translate them and returns a dictionary with the responses for each figure properly organized.
  """

  dict_numbers = {"I": 1, "II": 2,
                  "III": 3, "IV": 4,
                  "V": 5, "VI": 6,
                  "VII": 7, "VIII": 8,
                  "IX": 9, "X": 10}

  regex_response_number = '[A-Z]+/'                                # since the responses alway start with the number + \

  list_number_figure = re.findall(regex_response_number, raw_text)  # finds the regex for the number of responses

  text_i = re.split(regex_response_number, raw_text)

  # regex_choix_pos = r'Choix +(.*?)Choix -'

  # regex_choix_neg = r'Choix -(.*?)Rq :'

  # regex_rq = r'Rq :(.*?)\n'

  try: 
    
    text_choix_pos = re.findall(r'Choix \+ :(.*?)\nCh.?', raw_text)[0].strip()

    text_choix_neg = re.findall(r'Choix \- :(.*?)\n', raw_text)[0].strip()

    text_rq = re.findall(r'Rq :(.*?)\n', raw_text)[0].strip()

    additional_info = {"Choix_pos": text_choix_pos, "Choix_neg": text_choix_neg, "Rq": text_rq}
    
  except:
      
      additional_info = {"Choix_pos": "nada", "Choix_neg": "nada", "Rq": "nada"}
      
      print("Additional info not found")


  list_responses = []

  for i in range(len(list_number_figure)):

    if list_number_figure[i] not in text_i[i]:

      j = i + 1 # there is a \n\n string at index 0 so the text actually starts at index 1
      # print("not found")

    else:

      j = i

    dict_responses = {}

    n_response = list_number_figure[i][:-1]

    number = dict_numbers[n_response]

    text_i = re.split(regex_response_number, raw_text)

    full_text = text_i[j].strip()

    regex_line_break = '\n++'

    list_sentences_raw = text_i[j].split(".")

    text_i[j] = re.sub(regex_line_break, " ", text_i[j]) \
                  .replace("@", "") \
                  .replace("^", "") \
                  .replace("V", "") \
                  .replace(">", "") \
                  .replace("  ", " ") \
                  .strip() \

    # DEEPL API TRANSLATION

    with open("DEEPL_KEY\DEEPL_API_KEY.txt", "r") as f:
      API_KEY = f.read()

    translator = deepl.Translator(API_KEY)
    result = translator.translate_text(text_i[j], target_lang="EN-US", preserve_formatting=True)

    text_i[j] = result.text

    dict_responses["figure_number"] = number

    dict_responses["raw_response"] = full_text

    dict_responses["clean_response"] = text_i[j]

    list_sentences_clean = text_i[j].split(".")

    clean_sentences = []

    i = 0

    special_markers_list = ["@", "^", "V ", "<", ">",]

    # structuring the sentences inside the while loop

    while i < len(list_sentences_clean):

      if "Choix" in list_sentences_clean[i]:  # if we reach choix, there are no more actual responses by the patient, those are just comments by the psychologist
        break

      elif "Choice" in list_sentences_clean[i]:  # if we reach choix, there are no more actual responses by the patient, those are just comments by the psychologist
        break

      j = i + 1

      # cleaning the sentences and adding the . back at the end of the sentence

      dict_sentence_info = {}

      # use the full_text of the text here to parse, the text[j is clean
      item = list_sentences_clean[i].strip()

      item_2 = list_sentences_raw[i].strip()

      if item != "":                      # split leaves an empty string at the end of the list and by adding a . , we get a "." item at the end of the list
        item = item + "."
        dict_sentence_info["response_{}".format(j)] = item
        clean_sentences.append(dict_sentence_info)


        for marker in special_markers_list:
          if marker in item_2:
            dict_sentence_info["special_marker"] = marker

      j = j + 1

      i = i + 1

    dict_responses["sentences"] = clean_sentences
        # in case we need to double-check

    list_responses.append(dict_responses)

  list_responses.append(additional_info)



  return list_responses

<>:79: SyntaxWarning: invalid escape sequence '\D'
<>:79: SyntaxWarning: invalid escape sequence '\D'
C:\Users\alber\AppData\Local\Temp\ipykernel_12256\2486680214.py:79: SyntaxWarning: invalid escape sequence '\D'
  with open("DEEPL_KEY\DEEPL_API_KEY.txt", "r") as f:


In [7]:
responses = get_responses(raw_text)
responses

[{'figure_number': 1,
  'raw_response': '@ ^ Une sorte de coléoptère avec des grandes ailes\nou ça peut être une sorte de grenouille.\nV Une sorte de masque Viking avec quatre yeux.\nUne sorte de silhouette.\nUn petit personnage avec  une bouche ouverte qui a l’air surpris\net son reflet là.',
  'clean_response': 'Some kind of beetle with big wings, or it could be some kind of frog. Some kind of iking mask with four eyes. Some kind of silhouette. A small character with an open mouth who looks surprised and his reflection there.',
  'sentences': [{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'special_marker': '^'},
   {'response_2': 'Some kind of iking mask with four eyes.',
    'special_marker': 'V '},
   {'response_3': 'Some kind of silhouette.'},
   {'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'}]},
 {'figure_number': 2,
  'raw_response': 'V Deux personnages en train d’établir un contac

In [8]:
with open("demo_text_preprocess", "w") as f:
    json.dump(responses, f)

The proper clean single sentence responses in English are in the dictionary here: [{sentences: [{response_x}]}]

In [9]:
demo = responses
demo

[{'figure_number': 1,
  'raw_response': '@ ^ Une sorte de coléoptère avec des grandes ailes\nou ça peut être une sorte de grenouille.\nV Une sorte de masque Viking avec quatre yeux.\nUne sorte de silhouette.\nUn petit personnage avec  une bouche ouverte qui a l’air surpris\net son reflet là.',
  'clean_response': 'Some kind of beetle with big wings, or it could be some kind of frog. Some kind of iking mask with four eyes. Some kind of silhouette. A small character with an open mouth who looks surprised and his reflection there.',
  'sentences': [{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'special_marker': '^'},
   {'response_2': 'Some kind of iking mask with four eyes.',
    'special_marker': 'V '},
   {'response_3': 'Some kind of silhouette.'},
   {'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'}]},
 {'figure_number': 2,
  'raw_response': 'V Deux personnages en train d’établir un contac

# 2 Filtering the responses from the dictionary

This next couple of functions just take the previous dictionary and extracts just the sentences. It "shaves off" all the unnecesary fluff which will not be useful to process the text.

In [10]:
def get_list_figure_responses(og_dict):

    responses_list = []

    for figure in og_dict[:-1]:

        figure_sents = figure["sentences"]

        for item in figure_sents:

            item.pop("special_marker", None)
        responses_list.append(figure_sents)

    return responses_list

def clean_dict(responses_fig_list):

    '''merges the dictionary for each the responses of a figure into one response
    to go from a list of dicts [{response: blablabla},  {response: blablabla}, {response: blablabla}]
    to just one dict{response_1: , response_2: , response_3: }

    '''

    new_dict = {}
    for dictionary in responses_fig_list:
            for k , v in dictionary.items():
                # print(k,v)
                new_dict[k]= v
    return new_dict

# def figure_df(clean_dict):
#     '''gets the previous clean dict and turns it into a dataframe
#     '''
#     clean_responses_df = pd.DataFrame.from_dict(clean_dict, orient='index')
#     return clean_responses_df


In [11]:
def transform_dictionary_to_figure_list(og_dictionary):
    '''this function transforms the first dictionary
    into individual response dictionaries for each figure and returns it in list form.
    '''

    responses_list = get_list_figure_responses(og_dictionary)

    clean_fig_1 = clean_dict(responses_list[0])
    clean_fig_2 = clean_dict(responses_list[1])
    clean_fig_3 = clean_dict(responses_list[2])
    clean_fig_4 = clean_dict(responses_list[3])
    clean_fig_5 = clean_dict(responses_list[4])
    clean_fig_6 = clean_dict(responses_list[5])
    clean_fig_7 = clean_dict(responses_list[6])
    clean_fig_8 = clean_dict(responses_list[7])
    clean_fig_9 = clean_dict(responses_list[8])
    clean_fig_10 = clean_dict(responses_list[9])

    list_dicts = [clean_fig_1, clean_fig_2, clean_fig_3, clean_fig_4, clean_fig_5, clean_fig_6, clean_fig_7, clean_fig_8, clean_fig_9, clean_fig_10]

    return list_dicts

The output of the functions look something like this:

In [12]:
list_dicts = transform_dictionary_to_figure_list(demo)
list_dicts

[{'response_1': 'Some kind of beetle with big wings, or it could be some kind of frog.',
  'response_2': 'Some kind of iking mask with four eyes.',
  'response_3': 'Some kind of silhouette.',
  'response_4': 'A small character with an open mouth who looks surprised and his reflection there.'},
 {'response_1': "Two characters making contact or fighting because there's blood or a leg has been cut off.",
  'response_2': 'Lungs.',
  'response_3': "A rooster's head with a crest Fighter plane, glider, jet engine fire; again, it could be a larger one sending missiles Trunk men, perhaps irrational or some kind of caterpillar with a big belly."},
 {'response_1': 'Two oddly shaped guitars with twisted necks.',
  'response_2': 'Again, two figures face to face, strangely constituted, like Picasso.',
  'response_3': 'Or two figures as one.',
  'response_4': 'Two kinds of missiles or torpedoes.',
  'response_5': 'Lungs or a butterfly.',
  'response_6': 'Just part of an old man with his cane, or an e

In [13]:
with open("list_responses_clean.json", "w") as f:
    json.dump(list_dicts, f)

# Clean Noun Phrases (in progress, experimentation in Getting_noun_phrases.ipynb)

To mirror the input of the training data that will be use to train the model, instead of using the whole sentence, the noun phrases corresponding to each response will be extracted so that it can be fed to the model as the input to classify. Moreover, since there are many responses which actually contain more than one response, that is, several responses are coordinated with conjunctions, that will also be dealt with in this step.

In [14]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

In [15]:
def get_noun_phrase(doc):
    """Takes a NLP spaCy doc as an input and outputs the noun phrase subtree at the highest depth within the syntax tree
    """
    # print(doc)
    list_dependencies = ["nsubj", "pobj", "attr", "agent", "csubj", "csubjpass", "npmod", "oprd", "pobj", "dative", "appos", "ROOT"]
    pos = ["NOUN", "PROPN", "NUM"]
    
    trans_dictionary = {"[": "", 
                    "]": "", 
                    ",": "",
                    ".": "",
                    "  ": "",
                    " '": "'",
                    ".":",",
                    " )": ")",
                    "( ": "("}
    
    for token in doc:
        # print(token)


        if token.dep_ in list_dependencies and token.pos_ in pos and token.pos_ != "PRON":  # remove personal pronouns?
            ancestors = [t for t in token.ancestors if t.dep_ in list_dependencies and t.pos_ != "PRON" and t.dep_ !="ROOT"]
            
            # in the above code, verify that the "ROOT" dependency acts the way we intend
            
            # print(token,"ancestors:", ancestors)
            
            len_ancestors = len(ancestors)
            # print(ancestors, len_ancestors)
            if len_ancestors > 0 :
                continue
                
            elif len_ancestors == 0 :
                
                og_ancestor = token

                final_children = [t for t in og_ancestor.children]
                
                for child in final_children:
                    grand_children = [t for t in child.children] # this way we get all the tree under the og ancestor
                    for item in grand_children:
                        if item != None:
                            final_children.append(item)
                            
                final_children = final_children + [og_ancestor]  
                
                sorted_sentence = sorted(final_children) # if needed, we can access the token's index with token.i
                response = str(sorted_sentence).replace("[", "").replace("]", "").replace(",", "").replace("  ", " ").replace(".", "").replace(" )", ")").replace("( ", "(").strip(" ") #to clean the response
                # print(response)
                return response
                # sometimes the response will be empty
    return "no meaningful NP found"
    

In [16]:
def get_clean_NP_responses(list_of_dictionary_responses):
    """Takes a dictionary with responses as input 
    and outputs a dictionary with the noun phrases from those responses
    """
    clean_np_list = []
    
    for dictionary_response in list_of_dictionary_responses:
            
        clean_np_responses = {}

        for key, sentence in list(dictionary_response.items()):
            
            response_dict = {}
             
            # i will be using this to reintroduce sliced sentences into the dictionary at the end
            list_keys = list(dictionary_response.keys())
            last_key = list_keys[-1]
            key_template = str(last_key[:-1])+"{}"
            last_number = last_key[-1]

            doc = nlp(sentence)
            
            coordination = [(token, token.dep_, token.i) for token in doc if token.dep_ == "cc"] 
            
            if len(coordination) > 0 :  # slice the sentence by a token and add another response to the list
                
                # print(coordination)
                list_indexes = []
                
                for tuple in coordination:  # if there are more than one instances of coordination we'll need access to the indexes of every cc to know where to cut the sentences
                    index_cut = tuple[2]
                    list_indexes.append(index_cut)
                    
                new_sentences = []
                
                for i in range(len(list_indexes)):
                    
                    first_new_sentence = doc[:list_indexes[i]].text
                    try:
                        second_new_sentence = doc[list_indexes[i]+1:list_indexes[i+1]].text # will execute if there are more than one "cc"s
                    except:
                        second_new_sentence = doc[list_indexes[i]+1:].text
                    
                    new_sentences.append(first_new_sentence)
                    new_sentences.append(second_new_sentence)
                
                del dictionary_response[key]
                list_keys = list(dictionary_response.keys())
                last_key = list_keys[-1]
                key_template = str(last_key[:-1])+"{}"
                last_number = last_key[-1]
                
                for i in range(len(new_sentences)):
                    new_index_key = int(last_number) + i + 1 # since i will be 0 we have to add one so that we don't overwrite the last key
                    new_key = key_template.format(new_index_key)
            
                    doc = nlp(new_sentences[i])
                    # print(doc)
                    response_chopped_sentence = get_noun_phrase(doc)
                    np_key = len(list(clean_np_responses.keys())) + 1
                    clean_np_responses["response_{}".format(np_key)] = response_chopped_sentence
                    # response_dict[sentence] = response_chopped_sentence
                    
            elif len(coordination) == 0:
                # print(doc)
                response = get_noun_phrase(doc)
                np_key = len(list(clean_np_responses.keys())) + 1
                clean_np_responses["response_{}".format(np_key)] = response
                # response_dict[sentence] = response_chopped_sentence
                
        clean_np_list.append(clean_np_responses)
        
    return clean_np_list

Adapting the function for individual responses

In [17]:
def get_np(sentence):
    
    doc = nlp(sentence)

    coordination = [(token, token.dep_, token.i) for token in doc if token.dep_ == "cc"]
    
    if len(coordination) > 0 :  # slice the sentence by a token and add another response to the list
        
        # print(coordination)
        list_indexes = []
        
        for tuple in coordination:  # if there are more than one instances of coordination we'll need access to the indexes of every cc to know where to cut the sentences
            index_cut = tuple[2]
            list_indexes.append(index_cut)
            
        new_sentences = []
        
        for i in range(len(list_indexes)):
            
            first_new_sentence = doc[:list_indexes[i]].text
            try:
                second_new_sentence = doc[list_indexes[i]+1:list_indexes[i+1]].text # will execute if there are more than one "cc"s
            except:
                second_new_sentence = doc[list_indexes[i]+1:].text
            
            new_sentences.append(first_new_sentence)
            new_sentences.append(second_new_sentence)
            
        # print(new_sentences)
        
        for i in range(len(new_sentences)):

            doc = nlp(new_sentences[i])
            # print(doc)
            response_chopped_sentence = get_noun_phrase(doc)
            
            # print(response_chopped_sentence)
            
            new_sentences[i] = response_chopped_sentence
        
        coordination = True
        # print(new_sentences)
        return new_sentences, coordination

    elif len(coordination) == 0:
        # print(doc)
        response = get_noun_phrase(doc)
        
        coordination = False
        
        return response, coordination

In [18]:
clean_np_responses = get_clean_NP_responses(list_dicts)
clean_np_responses

[{'response_1': 'Some kind of beetle with big wings',
  'response_2': 'some kind of frog',
  'response_3': 'Some kind of iking mask with four eyes',
  'response_4': 'Some kind of silhouette',
  'response_5': 'A small character with an open mouth who looks surprised',
  'response_6': 'his reflection there'},
 {'response_1': 'Two characters',
  'response_2': 'blood',
  'response_3': "Two characters making contact or fighting because there 's blood",
  'response_4': 'no meaningful NP found',
  'response_5': 'Lungs',
  'response_6': "A rooster 's head with a crest Fighter plane glider jet engine fire",
  'response_7': 'some kind of caterpillar with a big belly'},
 {'response_1': 'Two',
  'response_2': 'two figures',
  'response_3': 'no meaningful NP found',
  'response_4': 'two figures as one',
  'response_5': 'Two kinds of missiles',
  'response_6': 'no meaningful NP found',
  'response_7': 'Lungs',
  'response_8': 'a butterfly',
  'response_9': 'Just part of an old man with his cane',
  

To account for repetitions, since it is also possible that the speaker repeats themselves and actually utters the same response more than once, a cosine similarity function will be implemented. As of now, an appropiate cutoff value for the similarity is not set so this functionality will not be in the prototype. However, the potential use of cosine similarity would be comparing the noun phrases we got from the previous section and deleting the sentence in the pair which has the shortest length because the longer sentence may contain a bit more information, even if just by a marginal amount (as accounted by the cosine similarity).

In [19]:
import itertools
from sentence_transformers import SentenceTransformer, util

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
def cosine_similarity_dict(dict):


    embeddigngs_model = SentenceTransformer("all-MiniLM-L6-v2") 

    list_cosine_similarity = []
    list_responses = []
    for key in dict:
        list_responses.append(dict[key])

    sentence_pairs = itertools.combinations(list_responses, 2)
    for pair in sentence_pairs:

        dictionary_cosine_similarity = {"pair": None, "cosine_similarity": None}

        # print(pair[0], "\n", pair[1])
        dictionary_cosine_similarity["pair"] = pair

        embedding_sentence_1 = embeddigngs_model.encode(pair[0], convert_to_tensor=True)
        embedding_sentence_2 = embeddigngs_model.encode(pair[1], convert_to_tensor=True)

        cosine_similarity_pairs = util.cos_sim(embedding_sentence_1, embedding_sentence_2)

        dictionary_cosine_similarity["cosine_similarity"] = round(float(cosine_similarity_pairs), 3)

        list_cosine_similarity.append(dictionary_cosine_similarity)

    return list_cosine_similarity

In [21]:
cosine_simil = cosine_similarity_dict(list_dicts[0])
cosine_simil

[{'pair': ('Some kind of iking mask with four eyes.',
   'Some kind of silhouette.'),
  'cosine_similarity': 0.457}]

# Setting up the evaluate function using the classification models 

In [22]:
import pickle
import numpy as np

In [23]:
# copied from the ML development notebook

def evaluate_one_vs_rest(path, text):
    
    pipeline = pickle.load(open(path, "rb"))
    
    if "content" in path:
        # print("content found")
        possible_outcomes = ['(A)', '(Ad)', '(H)', '(Hd)', 'A', 'Ad', 'An', 'Art', 'Ay', 'Bl', 'Bt',
       'Cg', 'Cl', 'Ex', 'Fd', 'Fi', 'Ge', 'H', 'Hd', 'Hh', 'Hx', 'Id', 'Ls',
       'Na', 'Sc', 'Sx', 'Xy']
        
    elif "determinant" in path:
        # print("determinant found")
        possible_outcomes = ['C', 'C\'', 'C\'F', 'CF', 'F', 'FC', 'FC\'', 'FD', 'FMa', 'FMa-p', 'FMp',
       'FT', 'FV', 'FY', 'Fr', 'Ma', 'Ma-p', 'Mp', 'T', 'TF', 'V', 'VF', 'Y',
       'YF', 'rF']

    prediction = pipeline.predict([text])
    probabilities = pipeline.predict_proba([text]) # sometimes no prediction is given back so we can take the outcome with the highest P instead

    # print("prediction:", prediction)
    # print("probabilities:", probabilities)
    
    list_predictions = prediction.tolist()
    list_predictions = [x for sublist in list_predictions for x in sublist] # avoid lists with sublists

    
    if len(list_predictions) != len(possible_outcomes): # sanity check
        print(prediction)
        print( len(list_predictions)  )
        print(possible_outcomes)
        print( len(possible_outcomes)  )
        print("Error encountered in the predictions")
        
    results = ([possible_outcomes[i] for i in range(len(list_predictions)) if list_predictions[i] == 1]) 

    if results == []:
        # print("No result")
        i = probabilities.argmax(1).item()
        # print(ix)
        final_results = possible_outcomes[i]
    
    else:
        final_results = str(results).replace("\'", "").replace("[", "").replace("]", "")
    
    return final_results

In [24]:
example = list_dicts[0]["response_2"]

In [25]:
# path_model_contents = r"Models\Contents\pipeline_contents_One-Many_V1-02-04.sav"


# evaluation = evaluate_one_vs_rest(path_model_contents, example)
# print(example)
# print(evaluation)

Some kind of iking mask with four eyes.
Hd


In [26]:
# path_model_determinants = r"Models\Determinants\pipeline_determinants_One-Many_V1-02-04.sav"


# evaluation = evaluate_one_vs_rest(path_model_determinants,example)
# print(example)
# print(evaluation)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
29
['C', "C'", "C'F", 'CF', 'F', 'FC', "FC'", 'FD', 'FMa', 'FMa-p', 'FMp', 'FT', 'FV', 'FY', 'Fr', 'Ma', 'Ma-p', 'Mp', 'T', 'TF', 'V', 'VF', 'Y', 'YF', 'rF']
25
Error encountered in the predictions
Some kind of iking mask with four eyes.
FC


# Adapting the functions to classify to the way we are storing our info

In [27]:
def eval(list_dicts, model_contents=r"Models\Contents\pipeline_contents_One-Many_V2-07-04.sav", model_determinants=r"Models\Determinants\pipeline_determinants_One-Many_V2-07-04.sav"):
    """This function runs the evaluation with our first two models. 
    It takes as input the list of dictionary responses, prints the evaluation 
    and returns the content and determinant labels for each response in dictionary form.   
    """
    evaluation = []
    i = 1
    for dictionary in list_dicts:
        
        figure_number = "Figure_{}".format(i)
        
        dict_evaluation_per_figure = {}
        
        print(figure_number, "\n") 
        
        list_evaluation_figure = []
        
        j = 1
        
        for response in dictionary:
            
            sentence = dictionary[response]
            # print(dictionary[response]) #works until here

            # print(response, content, determinant)
            
            dict_eval = {}
            dict_eval["response"] = sentence
            noun_phrases, coordination = get_np(sentence)
            dict_eval["noun_phrase"] = noun_phrases

            
            
            if noun_phrases == "no meaningful NP found":
                contents = "None"
                determinants = "None"
                dict_eval["content"] = contents
                dict_eval["determinant"] = determinants
            
            elif coordination == True:
                contents = []
                determinants = []
                for np in noun_phrases:
                    
                    print("Coordination Found!")
                    
                    content = evaluate_one_vs_rest(model_contents, np)
                    contents.append(content)
                    
                    determinant = evaluate_one_vs_rest(model_determinants, np)
                    determinants.append(determinant)
                    
                    dict_eval["content"] = contents
                    dict_eval["determinant"] = determinants
                    
                # print(response, content)
            elif coordination == False:
                # print(noun_phrases)
                contents = []
                determinants = []
                contents = evaluate_one_vs_rest(model_contents, noun_phrases)
                determinants = evaluate_one_vs_rest(model_determinants, noun_phrases)
                
                dict_eval["content"] = contents
                dict_eval["determinant"] = determinants
                
            dict_eval["position"] = None
            dict_eval["colour"] = None
            
            list_evaluation_figure.append(dict_eval)
            
            print("Response {}: ".format(j), sentence,"\nNoun Phrase(s):", noun_phrases, "\nContent:", contents , "\nDeterminant:", determinants, "\n")
            
            j = 1 + j
            
        dict_evaluation_per_figure[figure_number] = list_evaluation_figure
        
        evaluation.append(dict_evaluation_per_figure)
        
        i = i +1
        
    return evaluation

In [28]:
evaluation = eval(list_dicts)

Figure_1 



Response 1:  Some kind of iking mask with four eyes. 
Noun Phrase(s): Some kind of iking mask with four eyes 
Content: Hd 
Determinant: F 

Response 2:  Some kind of silhouette. 
Noun Phrase(s): Some kind of silhouette 
Content: Sc 
Determinant: F 

Figure_2 

Response 1:  Lungs. 
Noun Phrase(s): Lungs 
Content: An 
Determinant: FY 

Figure_3 

Response 1:  Two oddly shaped guitars with twisted necks. 
Noun Phrase(s): Two 
Content: H, Xy 
Determinant: CF 

Response 2:  Again, two figures face to face, strangely constituted, like Picasso. 
Noun Phrase(s): two figures 
Content: H 
Determinant: FC 

Response 3:  Some kind of big beast, you can see the inside of its body, the mandibles. 
Noun Phrase(s): big beast 
Content: A 
Determinant: F 

Figure_4 

Response 1:  Two figures under a kind of burka. 
Noun Phrase(s): Two figures under a kind of burka 
Content: H 
Determinant: Ma 

Response 2:  The fly's head (film). 
Noun Phrase(s): The fly 's head (film) 
Content: (Hd) 
Determinant: F 

F

In [29]:
evaluation

[{'Figure_1': [{'response': 'Some kind of iking mask with four eyes.',
    'noun_phrase': 'Some kind of iking mask with four eyes',
    'content': 'Hd',
    'determinant': 'F',
    'position': None,
    'colour': None},
   {'response': 'Some kind of silhouette.',
    'noun_phrase': 'Some kind of silhouette',
    'content': 'Sc',
    'determinant': 'F',
    'position': None,
    'colour': None}]},
 {'Figure_2': [{'response': 'Lungs.',
    'noun_phrase': 'Lungs',
    'content': 'An',
    'determinant': 'FY',
    'position': None,
    'colour': None}]},
 {'Figure_3': [{'response': 'Two oddly shaped guitars with twisted necks.',
    'noun_phrase': 'Two',
    'content': 'H, Xy',
    'determinant': 'CF',
    'position': None,
    'colour': None},
   {'response': 'Again, two figures face to face, strangely constituted, like Picasso.',
    'noun_phrase': 'two figures',
    'content': 'H',
    'determinant': 'FC',
    'position': None,
    'colour': None},
   {'response': 'Some kind of big beas

# 4 PUTTING IT ALL TOGETHER

Finally, putting the functions together to generate a mini-report and the corresponding results dictionary

In [30]:
def raw_text_response_eval(raw_text):
    
    responses = get_responses(raw_text)

    list_dicts = transform_dictionary_to_figure_list(responses)

    evaluation = eval(list_dicts)
    
    return evaluation

In [31]:
def translated_dict_response_eval(dictionary): 
    """Use this one if you are working with the demo dictionaries already done to save deepl credits"""
    
    responses = dictionary

    list_dicts = transform_dictionary_to_figure_list(responses)

    evaluation = eval(list_dicts)
    
    return evaluation

In [32]:
# raw_text_response_eval(raw_text)

In [33]:
evaluation = translated_dict_response_eval(demo) 

Figure_1 

Coordination Found!


Coordination Found!
Response 1:  Some kind of beetle with big wings, or it could be some kind of frog. 
Noun Phrase(s): ['Some kind of beetle with big wings', 'some kind of frog'] 
Content: ['A', 'A'] 
Determinant: ['Ma', 'FMp'] 

Response 2:  Some kind of iking mask with four eyes. 
Noun Phrase(s): Some kind of iking mask with four eyes 
Content: Hd 
Determinant: F 

Response 3:  Some kind of silhouette. 
Noun Phrase(s): Some kind of silhouette 
Content: Sc 
Determinant: F 

Coordination Found!
Coordination Found!
Response 4:  A small character with an open mouth who looks surprised and his reflection there. 
Noun Phrase(s): ['A small character with an open mouth who looks surprised', 'his reflection there'] 
Content: ['Id', 'A'] 
Determinant: ['F', 'F'] 

Figure_2 

Coordination Found!
Coordination Found!
Coordination Found!
Coordination Found!
Response 1:  Two characters making contact or fighting because there's blood or a leg has been cut off. 
Noun Phrase(s): ['Two characters', '

In [34]:
evaluation

[{'Figure_1': [{'response': 'Some kind of beetle with big wings, or it could be some kind of frog.',
    'noun_phrase': ['Some kind of beetle with big wings', 'some kind of frog'],
    'content': ['A', 'A'],
    'determinant': ['Ma', 'FMp'],
    'position': None,
    'colour': None},
   {'response': 'Some kind of iking mask with four eyes.',
    'noun_phrase': 'Some kind of iking mask with four eyes',
    'content': 'Hd',
    'determinant': 'F',
    'position': None,
    'colour': None},
   {'response': 'Some kind of silhouette.',
    'noun_phrase': 'Some kind of silhouette',
    'content': 'Sc',
    'determinant': 'F',
    'position': None,
    'colour': None},
   {'response': 'A small character with an open mouth who looks surprised and his reflection there.',
    'noun_phrase': ['A small character with an open mouth who looks surprised',
     'his reflection there'],
    'content': ['Id', 'A'],
    'determinant': ['F', 'F'],
    'position': None,
    'colour': None}]},
 {'Figure_2':